In [1]:
import pandas as pd

In [2]:
ssj = pd.read_table("miniPCAWG.ssj.tsv.gz", header=None)

In [4]:
ssj.columns = ["junction_id", "offset", "F1", "R1", "F2", "R2"]

In [5]:
ssj

,junction_id,offset,F1,R1,F2,R2
0,22_16151821_16162397,7,0,2,0,0
1,22_16151821_16162397,14,0,0,1,0
2,22_16151821_16162397,18,0,0,0,1
3,22_16151821_16162397,25,0,0,0,1
4,22_16151821_16162397,35,1,0,0,0
...,...,...,...,...,...,...
162158,22_51227226_51227323,50,0,0,1,0
162159,22_51227226_51227323,51,0,0,1,0
162160,22_51227226_51227323,81,0,0,1,0
162161,22_51227230_51227320,64,1,0,0,0


In [16]:
i = 0
for x in ssj.itertuples():
    print(x)
    i += 1
    if i > 10:
        break

Pandas(Index=0, junction_id='22_16151821_16162397', offset=7, F1=0, R1=2, F2=0, R2=0)
Pandas(Index=1, junction_id='22_16151821_16162397', offset=14, F1=0, R1=0, F2=1, R2=0)
Pandas(Index=2, junction_id='22_16151821_16162397', offset=18, F1=0, R1=0, F2=0, R2=1)
Pandas(Index=3, junction_id='22_16151821_16162397', offset=25, F1=0, R1=0, F2=0, R2=1)
Pandas(Index=4, junction_id='22_16151821_16162397', offset=35, F1=1, R1=0, F2=0, R2=0)
Pandas(Index=5, junction_id='22_16151821_16162397', offset=71, F1=0, R1=0, F2=2, R2=0)
Pandas(Index=6, junction_id='22_16151821_16162397', offset=75, F1=1, R1=0, F2=0, R2=0)
Pandas(Index=7, junction_id='22_16151821_16162397', offset=84, F1=0, R1=1, F2=0, R2=0)
Pandas(Index=8, junction_id='22_16159389_16162397', offset=1, F1=0, R1=0, F2=1, R2=0)
Pandas(Index=9, junction_id='22_16159389_16162397', offset=25, F1=1, R1=0, F2=0, R2=0)
Pandas(Index=10, junction_id='22_16159389_16162397', offset=90, F1=0, R1=0, F2=0, R2=1)


In [19]:
from collections import namedtuple

In [31]:
NewRow = namedtuple("Pandas", ["junction_id", "offset", "count"])

In [34]:
def update_ssj(df, read_length, min_offset, stranded):
    max_offset = read_length - min_offset
    rows_list = []
    i = 0
    if stranded:
        for row in df.itertuples():
            if row.offset < min_offset or row.offset > max_offset:
                continue
            sm = row.F1 + row.R2
            sp = row.F2 + row.R1
            if sp > 0:
                rows_list.append(NewRow(junction_id=row.junction_id+"_+", offset=row.offset, counts = sp))
                i += 1
            if sm > 0:
                rows_list.append(NewRow(junction_id=row.junction_id+"_-", offset=row.offset, counts = sm))
                i += 1
    else:
        for row in df.itertuples():
            if row.offset < min_offset or row.offset > max_offset:
                continue
            s = row.F1 + row.R1 + row.F2 + row.R2
            rows_list.append(NewRow(junction_id=row.junction_id+"_+", offset=row.offset, count=s))
            i += 1
            rows_list.append(NewRow(junction_id=row.junction_id+"_-", offset=row.offset, count=s))
            i += 1
    return pd.DataFrame(rows_list, columns=["junction_id", "offset", "count"])

In [39]:
nssj = update_ssj(ssj, read_length=101, min_offset=5, stranded=False)

In [40]:
nssj

,junction_id,offset,count
0,22_16151821_16162397_+,7,2
1,22_16151821_16162397_-,7,2
2,22_16151821_16162397_+,14,1
3,22_16151821_16162397_-,14,1
4,22_16151821_16162397_+,18,1
...,...,...,...
296433,22_51227226_51227323_-,81,1
296434,22_51227230_51227320_+,64,1
296435,22_51227230_51227320_-,64,1
296436,22_51227230_51227320_+,87,1


In [41]:
import numpy as np

In [44]:
x = np.array([25, 12, 4])

In [48]:
np.log2(x.sum()) - (np.sum(x * np.log2(x)) / x.sum())

1.281553108202914

In [49]:
test_ssj = pd.DataFrame([
    NewRow("chr1_100_200_-", 10, 25),
    NewRow("chr1_100_200_-", 11, 12),
    NewRow("chr1_100_200_-", 15, 4)
])

In [57]:
test_ssj.groupby(["junction_id"]).agg({
    "count": ["sum", "count", lambda x: np.log2(x.sum()) - (np.sum(x * np.log2(x)) / x.sum())]
})

count                 
                 sum count <lambda_0>
junction_id                          
chr1_100_200_-    41     3   1.281553

In [65]:
def aggregate_ssj(ssj):
    agg_ssj = ssj.groupby(["junction_id"]).agg(
        total_count=pd.NamedAgg(column="count", aggfunc=sum),
        staggered_count=pd.NamedAgg(column="count", aggfunc="count"),
        entropy=pd.NamedAgg(column="count", aggfunc=lambda x: np.log2(x.sum()) - (np.sum(x * np.log2(x)) / x.sum()))
    )
    return agg_ssj

In [67]:
assj = aggregate_ssj(nssj)

In [71]:
assj.to_csv("miniPCAWG.aggssj.tsv.gz", sep="\t", header=None, compression="gzip")

In [74]:
i = 0
for x in assj.index:
    print(x)
    i += 1
    if i > 10:
        break

22_16151821_16162397_+
22_16151821_16162397_-
22_16159389_16162397_+
22_16159389_16162397_-
22_16162487_16186811_+
22_16162487_16186811_-
22_16186946_16187165_+
22_16186946_16187165_-
22_16187278_16189032_+
22_16187278_16189032_-
22_16187302_16189032_+


In [75]:
import pysam

In [78]:
genome = pysam.FastaFile("/home/leo/PycharmProjects/pyIPSA/genomes/hg19.fa")

In [79]:
genome.fetch("chr22", 11579, 11580)

'N'

In [88]:
complement = {"A": "T", "G": "C", "C": "G", "T": "A", "N": "N"}
nucs = []
for junction_id in assj.index:
    ref, start, stop, strand = junction_id.split("_")
    ref, start, stop = "chr" + ref, int(start), int(stop)
    c = strand == "-"
    nucleotides = genome.fetch(ref, start, start + 2) + genome.fetch(ref, stop - 3, stop - 1)
    if c:
        nucleotides = "".join(complement[n] for n in nucleotides)
    nucs.append(nucleotides)

In [91]:
from collections import Counter

In [93]:
assj["splice_sites"] = nucs

In [94]:
assj

,total_count,staggered_count,entropy,splice_sites
junction_id,,,,
22_16151821_16162397_+,10,8,2.921928,CTAC
22_16151821_16162397_-,10,8,2.921928,GATG
22_16159389_16162397_+,2,2,1.000000,CTAC
22_16159389_16162397_-,2,2,1.000000,GATG
22_16162487_16186811_+,9,9,3.169925,CTAC
...,...,...,...,...
22_51227226_51227320_-,2,2,1.000000,CATC
22_51227226_51227323_+,8,6,2.500000,GTAG
22_51227226_51227323_-,8,6,2.500000,CATC
